<a href="https://colab.research.google.com/github/rakijja/Projekt_klasyfikacja_nadzorowana/blob/main/Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Wstęp

In [3]:
# import bibliotek
import numpy as np
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

from google.colab import files
import io

In [78]:
# ustawienie ziarna
seed = 123

np.random.seed(seed)

In [163]:
# wczytywanie danych
uploaded = files.upload()

Saving X_test.csv to X_test.csv


## Przetwarzanie danych

### Zbiór treningowy

In [343]:
# przypisanie zmiennych
df_1 = pd.read_csv("df_1.csv")
df_2 = pd.read_csv("df_2.csv")
target = pd.read_csv("target.csv")

In [344]:
# łączenie tabel
df_3 = df_1.merge(df_2, how='inner', on='EmployeeNumber')
df = df_3.merge(target, how='inner', on='EmployeeNumber')
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0_y', "Unnamed: 0_x", "Over18", "StandardHours"])

In [345]:
# usuwanie szumów
df = df[(df["TotalWorkingYears"]<100)]
df = df[(df["Age"]<100)]
df = df[(df["YearsAtCompany"]<100)]
df = df[(df["YearsInCurrentRole"]<100)]
df = df[(df["YearsSinceLastPromotion"]<100)]
df = df[(df["YearsWithCurrManager"]<100)]
df = df[(df["DistanceFromHome"]<20000)]

In [346]:
# encodowanie OHE
df_ohe = pd.get_dummies(df)

In [347]:
# binaryzacja
df_ohe = df_ohe.drop(columns=["Attrition_No", "OverTime_No", "Gender_Female"])

### Zbiór testowy

In [348]:
# przypisanie zmiennych
X_test = pd.read_csv("X_test.csv")

In [349]:
# usuwanie zbędnych kolumn
X_test = X_test.drop(columns=["Over18", "StandardHours"])

In [350]:
# encodowanie OHE
X_test_ohe = pd.get_dummies(X_test)

In [351]:
# binaryzacja
X_test_ohe = X_test_ohe.drop(columns=["OverTime_No", "Gender_Female"])

## Modelowanie

In [352]:
# przypisanie zmiennych
X_train = df_ohe.drop(columns=["Attrition_Yes"]) # Zmienne niezależne
y_train = df_ohe["Attrition_Yes"] # Zmienna zależna

In [296]:
# podział na zbior testowy i treningowy w celach testowych
"""
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y, random_state = seed)
"""

### Random Forest

In [390]:
clf = RandomForestClassifier(n_estimators = 100)

clf.fit(X_train, y_train)
y_pred_forest = clf.predict(X_test_ohe)

In [261]:
# raport z klasyfikacji
"""
print(classification_report(y_test, y_pred_forest, labels = [0, 1], digits = 4))
"""

              precision    recall  f1-score   support

           0     0.7490    0.8506    0.7966       435
           1     0.6934    0.5424    0.6087       271

    accuracy                         0.7323       706
   macro avg     0.7212    0.6965    0.7026       706
weighted avg     0.7276    0.7323    0.7244       706



### Decision Tree

In [243]:
tree = DecisionTreeClassifier(random_state = seed)

tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test_ohe)

In [244]:
# raport z klasyfikacji
"""
print(classification_report(y_test, y_pred_tree, labels = [0, 1], digits = 4))
"""

              precision    recall  f1-score   support

           0     0.7517    0.7655    0.7585       435
           1     0.6122    0.5941    0.6030       271

    accuracy                         0.6997       706
   macro avg     0.6819    0.6798    0.6808       706
weighted avg     0.6981    0.6997    0.6988       706



### Bagging

In [245]:
bclf = BaggingClassifier(n_estimators = 50, random_state = seed)
bclf.fit(X_train, y_train)

y_pred_bagging = bclf.predict(X_test_ohe)

In [246]:
# raport z klasyfikacji
"""
print(classification_report(y_test, y_pred_bagging, labels = [0, 1], digits = 4))
"""

              precision    recall  f1-score   support

           0     0.7735    0.8322    0.8018       435
           1     0.6933    0.6089    0.6483       271

    accuracy                         0.7465       706
   macro avg     0.7334    0.7205    0.7251       706
weighted avg     0.7427    0.7465    0.7429       706



### AdaBoost

In [376]:
clf = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth = 2), n_estimators = 10, learning_rate = 0.01)

clf.fit(X_train, y_train)

y_pred_adaboost = clf.predict(X_test_ohe)

In [298]:
# raport z klasyfikacji
"""
print(classification_report(y_test, y_pred_adaboost, labels = [0, 1], digits = 4))
"""

              precision    recall  f1-score   support

           0     0.8204    0.7770    0.7981       435
           1     0.6701    0.7269    0.6973       271

    accuracy                         0.7578       706
   macro avg     0.7452    0.7520    0.7477       706
weighted avg     0.7627    0.7578    0.7594       706



## Kroswalidacja

In [373]:
param_grid = {
    "n_estimators" : [5, 10, 100, 200, 500, 1000],
    "learning_rate" : [0.0001, 0.001, 0.01, 0.1, 1]
}


from sklearn.model_selection import GridSearchCV

cv = GridSearchCV(estimator = clf, param_grid = param_grid, scoring = "f1", cv = 5)
cv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2),
                                          learning_rate=0.01,
                                          n_estimators=1000),
             param_grid={'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1],
                         'n_estimators': [5, 10, 100, 200, 500, 1000]},
             scoring='f1')

In [374]:
print(f"Best parameters: {cv.best_params_}")

Best parameters: {'learning_rate': 0.01, 'n_estimators': 10}


## Predykcja

In [391]:
# przypisanie wyników predykcji
# y_pred_best = y_pred_forest
y_pred_best = y_pred_adaboost

In [392]:
# stworzenie nowej kolumny predykcji
# 0 - został w pracy, 
# 1 - odszedł z pracy
X_test["prediction"] = y_pred_best

map_class = {
    0 : 0,
    1 : 1
}

In [393]:
# mapowanie zmiennej
X_test["prediction"] = X_test["prediction"].map(map_class)

In [395]:
# utworzenie tabeli wynikowej
X_test_pred_rf_2 = X_test[['EmployeeNumber', 'prediction']]
X_test_pred_rf_2

In [396]:
# zapisanie tabeli wynikowej do nowego pliku
X_test_pred_rf_2.to_csv("X_test_pred_rf_2.csv")